# BLIMP-style benchmark for the Dyck and Dyck-u languages

In [1]:
import random
from nltk import Tree
dataset = "data/dyckkm/dyckkm_k64_m7_100000000.dev.json"

In [3]:
from nltk import CFG, ChartParser

def make_dycku_grammar():

    rules_basic = []
    rules_basic.append("S -> N V")
    rules_basic.append("S -> NSg V")
    rules_basic.append("S -> NSg VSg")
    rules_basic.append("S -> NPl V")
    rules_basic.append("S -> NPl VPl")
    rules_basic.append("S -> N VSg")
    rules_basic.append("S -> N VPl")
    rules_basic.append("S -> N S V")
    rules_basic.append("S -> NSg S VSg")
    rules_basic.append("S -> NPl S VPl")
    rules_basic.append("S -> NSg S V")
    rules_basic.append("S -> NPl S V")
    rules_basic.append("S -> N S VSg")
    rules_basic.append("S -> N S VPl")
    rules_basic.append("N -> 'n'")
    rules_basic.append("NSg -> 'nsg'")
    rules_basic.append("NPl -> 'npl'")
    rules_basic.append("V -> 'v'")
    rules_basic.append("VSg -> 'vsg'")
    rules_basic.append("VPl -> 'vpl'")

    rules_conj = [r + " S" for r in rules_basic if r.startswith("S")] # version with many rules but no structural ambiguity
    #rules_conj = ["S -> S S"] # less rules but many parses

    rules = rules_basic + rules_conj
    grammar = CFG.fromstring("\n".join(rules))
    parser = ChartParser(grammar)
    gr = parser.grammar() # grammar but slightly different. 
    return gr, parser

import string
def get_identifier_iterator():
  """ Returns an iterator to provide unique ids to bracket types.
  """
  ids = iter(list(string.ascii_lowercase))
  k = 1
  while True:
    try:
      str_id = next(ids)
    except StopIteration:
      ids = iter(list(string.ascii_lowercase))
      k += 1
      str_id = next(ids)
    yield str_id*k

def get_vocab_of_bracket_types(bracket_types):
  """ Returns the vocabulary corresponding to the number of brackets.

  There are bracket_types open brackets, bracket_types close brackets,
  START, and END.
  Arguments:
    bracket_types: int (k in Dyck-(k,m))
  Returns:
    Dictionary mapping symbol string  s to int ids.
  """
  id_iterator = get_identifier_iterator()
  ids = [next(id_iterator) for x in range(bracket_types)]
  vocab = {x: c for c, x in enumerate(['(' + id_str for id_str in ids] + [id_str + ')' for id_str in ids] + ['START', 'END'])}
  return vocab, ids

def make_dyck1_grammar():
    rules_basic = []
    rules_basic.append("S -> '<a' 'a>'")
    rules_basic.append("S -> '<a' S 'a>'")
    #rules_basic.append("S -> '<a' 'a>' S")
    rules_conj = [r + " S" for r in rules_basic if r.startswith("S")] # version with many rules but no structural ambiguity
    #rules_conj = ["S -> S S"] # less rules but many parses

    rules = rules_basic + rules_conj
    grammar = CFG.fromstring("\n".join(rules))
    parser = ChartParser(grammar)
    gr = parser.grammar() # grammar but slightly different. 
    return gr, parser
_, dyck1_parser = make_dyck1_grammar()
def make_dyck2_grammar():
    rules_basic = []
    lits = ["a", "b"]
    for lit in lits:
        rules_basic.append(f"S -> '<{lit}' '{lit}>'")
        rules_basic.append(f"S -> '<{lit}' S '{lit}>'")
    #rules_basic.append("S -> '<a' 'a>' S")
    rules_conj = [r + " S" for r in rules_basic if r.startswith("S")] # version with many rules but no structural ambiguity
    #rules_conj = ["S -> S S"] # less rules but many parses

    rules = rules_basic + rules_conj
    grammar = CFG.fromstring("\n".join(rules))
    parser = ChartParser(grammar)
    gr = parser.grammar() # grammar but slightly different. 
    return gr, parser

def make_dyck64_grammar():
    rules_basic = []
    for lit in get_vocab_of_bracket_types(64)[1]:
        rules_basic.append(f"S -> '<{lit}' '{lit}>'")
        rules_basic.append(f"S -> '<{lit}' S '{lit}>'")
    #rules_basic.append("S -> '<a' 'a>' S")
    rules_conj = [r + " S" for r in rules_basic if r.startswith("S")] # version with many rules but no structural ambiguity
    #rules_conj = ["S -> S S"] # less rules but many parses

    rules = rules_basic + rules_conj
    grammar = CFG.fromstring("\n".join(rules))
    parser = ChartParser(grammar)
    gr = parser.grammar() # grammar but slightly different. 
    return gr, parser

def make_dyck_parser(k):
    if k==1:
       return make_dyck1_grammar()[1]
    if k==2:
       return make_dyck2_grammar()[1]
    if k==64:
       return make_dyck64_grammar()[1]
_, dyck1_parser = make_dyck1_grammar()


In [4]:
from silm.gpst.eval_utils import load_gold_data
import tqdm

span_to_create = 2

def create_blimp_data_for_file(dataset, span_to_create, parser):
    to_skip = []
    texts, trees = load_gold_data(dataset, bos_tok="", eos_tok="")
    for sent_ix,(tree, tokens) in tqdm.tqdm(enumerate(zip(trees, texts))):
        tokens = tokens.split()
        assert tree.leaves() == tokens
        success = False
        ixs_to_try = list(range(len(tree.leaves())))
        random.shuffle(ixs_to_try)
        for chosen_index in ixs_to_try:        
            #chosen_index = random.choice(range(len(tree.leaves())))
            chosen_token = tokens[chosen_index]

            leaf_tp = tree.leaf_treeposition(chosen_index)
            if leaf_tp[-1] == 0:
                subtrees = [tree] # relevant subtree
                for i in leaf_tp[:-1]:
                    subtrees.append(subtrees[-1][i])
                relevant_subtree = subtrees[-1]
                span = len(relevant_subtree.leaves())
                closing_token = f"{chosen_token[1:]}>"
                if span==2 == span_to_create == 2: # ( ) case
                    relevant_subtree[0] = closing_token
                    relevant_subtree[1] = chosen_token
                    #for subtree, i in zip(subtrees[-2::-1], leaf_tp[-2::-1]):
                    #    subtree[i] = relevant_subtree
                    #    relevant_subtree = subtree[i]
                    if len(list(parser.parse(tree.leaves()))) == 0:
                        success = True
                    
                if span==span_to_create: # ( ( ) ) case
                    relevant_subtree[0] = closing_token
                    relevant_subtree[-1] = chosen_token
                    #for subtree, i in zip(subtrees[-2::-1], leaf_tp[-2::-1]):
                    #    subtree[i] = relevant_subtree
                    #    relevant_subtree = subtree[i]
                    if len(list(parser.parse(tree.leaves()))) == 0:
                        success = True
                    
                if success:
                    break
        if not success:
            to_skip.append(sent_ix)
    good, bad = [], []
    for sent_ix, (tree, text) in enumerate(zip(trees, texts)):
        if sent_ix not in to_skip:
            good.append(text)
            bad.append(" ".join(tree.leaves()))
    return good, bad 

In [13]:
ks = [1,2,64]
splits = ["dev","test"]
spans = [3,12] #[1,48,64]# [16,24,32] # [2,4,6,8,10]
datasets = []

import json, random
blimp_datasets = []
for k in ks:
    parser = make_dyck_parser(k)
    for split in splits:
        dataset = f"data/dyckkm/dyckkm_k{k}_m7_100000000.{split}.json"
        
        for span in spans: 
            print(k, span, split)
            good, bad = create_blimp_data_for_file(dataset, span, parser)
            with open(f"data/blimpfordyck/dyck_k{k}/dyck_k{k}_{split}_blimpdyck_{span}.jsonl", "w") as f:
                for g,b in zip(good, bad):
                    f.write(json.dumps({
                        "sentence_good": g,
                        "sentence_bad": b
                    }))
                    f.write("\n")

1 3 dev


3925it [00:00, 4685.29it/s]


1 12 dev


3925it [00:02, 1424.62it/s]


1 3 test


...

3946it [00:01, 1997.42it/s]


64 3 test


2750it [00:01, 2053.41it/s]


64 12 test


2750it [00:02, 941.12it/s] 


In [12]:
import json
import random
# same for Dyck-u
splits = ["dev","test"]
spans = [3,12]# [1,48,64]# [16,24,32] # [2,4,6,8,10]
datasets = []

from silm.gpst.eval_utils import load_gold_data
import tqdm

def create_blimp_data_for_dycku_file(dataset, span_to_create):
    to_skip = []
    parser = make_dycku_grammar()[1]
    texts, trees = load_gold_data(dataset, bos_tok="", eos_tok="")
    for sent_ix,(tree, tokens) in tqdm.tqdm(enumerate(zip(trees, texts))):
        tokens = tokens.split()
        assert tree.leaves() == tokens
        success=False
        ixs_to_try = list(range(len(tree.leaves())))
        random.shuffle(ixs_to_try)
        for chosen_index in ixs_to_try: 
            chosen_index = random.choice(range(len(tree.leaves())))
            chosen_token = tokens[chosen_index]

            leaf_tp = tree.leaf_treeposition(chosen_index)
            if leaf_tp[-1] == 0:
                subtrees = [tree] # relevant subtree
                for i in leaf_tp[:-2]:
                    subtrees.append(subtrees[-1][i])
                relevant_subtree = subtrees[-1]
                span = len(relevant_subtree.leaves())
                closing_token = f"{chosen_token[1:]}>"
                if span==2 == span_to_create == 2: # ( ) case
                    #relevant_subtree[0] = closing_token
                    #relevant_subtree[1] = chosen_token
                    opening_subtree = relevant_subtree[0]
                    closing_subtree = relevant_subtree[1]
                    relevant_subtree[0] = closing_subtree
                    relevant_subtree[1] = opening_subtree
                    #for subtree, i in zip(subtrees[-2::-1], leaf_tp[-2::-1]):
                    #    subtree[i] = relevant_subtree
                    #    relevant_subtree = subtree[i]
                    if len(list(parser.parse(tree.leaves()))) == 0:
                        success = True
                    #break
                elif span==span_to_create: # ( ( ) ) case
                    opening_subtree = relevant_subtree[0]
                    if len(relevant_subtree[-1].leaves())==1: 
                        closing_subtree = relevant_subtree[-1]
                        relevant_subtree[0] = closing_subtree
                        relevant_subtree[-1] = opening_subtree
                        #for subtree, i in zip(subtrees[-2::-1], leaf_tp[-2::-1]):
                        #    subtree[i] = relevant_subtree
                        #    relevant_subtree = subtree[i]
                        if len(list(parser.parse(tree.leaves()))) == 0:
                            success = True
                        #break
                if success:
                    break
                
        if not success:
            to_skip.append(sent_ix)
    good, bad = [], []
    for sent_ix, (tree, text) in enumerate(zip(trees, texts)):
        if sent_ix not in to_skip:
            good.append(text)
            bad.append(" ".join(tree.leaves()))
    return good, bad 

import json
blimp_datasets = []
for split in splits:
    dataset = f"data/dyckkm/underspec_100000000.{split}.json"
    
    for span in spans: 
        print(span, split)
        good, bad = create_blimp_data_for_dycku_file(dataset, span)
        with open(f"data/blimpfordyck/dyck_u/dyck_u_{split}_blimpdyck_{span}.jsonl", "w") as f:
            for g,b in zip(good, bad):
                f.write(json.dumps({
                    "sentence_good": g,
                    "sentence_bad": b
                }))
                f.write("\n")

3 dev


4132it [00:02, 1670.43it/s]


12 dev


4132it [00:05, 721.60it/s]


3 test


2485it [00:04, 518.65it/s]


12 test


2485it [00:10, 245.68it/s]


In [32]:
len(good), 4132

(1244, 4132)

In [33]:
good[1], bad[1]

('n n npl v nsg n vsg nsg nsg v v n vpl n v nsg nsg npl npl npl nsg v vpl v v npl vpl vsg npl nsg n npl vpl v vsg v v vsg vpl npl n v vpl npl nsg npl vpl vsg vpl vsg n nsg v vsg',
 'n n npl v nsg n vsg nsg nsg v v n vpl n v nsg nsg npl npl npl nsg v vpl v v npl vpl vsg v nsg n npl vpl v vsg npl v vsg vpl npl n v vpl npl nsg npl vpl vsg vpl vsg n nsg v vsg')

## Other data types

- `type_mismatch` Replacing brackets with brackets from other bracket types (different numbers of brackets, closing vs. opening)
- `break_bracketing` Swapping randomly chosen brackets (with different linear distances)

In [ ]:
# type_mismatch: 
dataset = 'data/dyckkm/dyckkm_k64_m7_100000000.dev.json'
texts, trees = load_gold_data(dataset, bos_tok="", eos_tok="")

In [17]:
parser = make_dyck_parser(64)
vocabulary = get_vocab_of_bracket_types(64)[1]

In [5]:
def create_type_mismatch_data(num_replacements, texts, parser, vocabulary):
    results = []
    for sent in tqdm.tqdm(texts):
        success = False
        if len(sent.split()) < num_replacements:
            continue
        while not success:
            replaced_ixs = set()
            sentsplit = sent.split()
            for i in range(num_replacements):
                ix_to_replace = random.choice(list(set(range(len(sentsplit))) - replaced_ixs))
                old_token = sentsplit[ix_to_replace]
                opening = old_token.startswith("<")
                if opening:
                    new_token = f"<{random.choice(vocabulary)}"
                else:
                    new_token = f"{random.choice(vocabulary)}>"

                sentsplit[ix_to_replace] = new_token
                replaced_ixs.add(ix_to_replace)

            if len(list(parser.parse(sentsplit))) == 0:
                success = True
                results.append((sent, ' '.join(sentsplit)))
    return [res[0] for res in results], [res[1] for res in results]

def create_type_mismatch_u_data(num_replacements, texts):
    
    parser = make_dycku_grammar()[1]
    results = []

    max_tries = 100
    for sent in tqdm.tqdm(texts):
        success = False
        num_tries = 0
        if len(sent.split()) < num_replacements:
            continue
        vocabulary = ["", "sg", "pl"]
        while not success:
            replaced_ixs = set()
            sentsplit = sent.split()
            for i in range(num_replacements):
                ix_to_replace = random.choice(list(set(range(len(sentsplit))) - replaced_ixs))
                old_token = sentsplit[ix_to_replace]
                opening = old_token.startswith("n")
                if opening:
                    new_token = f"n{random.choice(vocabulary)}"
                else:
                    new_token = f"v{random.choice(vocabulary)}"

                sentsplit[ix_to_replace] = new_token
                replaced_ixs.add(ix_to_replace)

            if len(list(parser.parse(sentsplit))) == 0:
                success = True
                results.append((sent, ' '.join(sentsplit)))
            if not success:
                num_tries += 1
                if num_tries >= max_tries:
                    success=True 
    return [res[0] for res in results], [res[1] for res in results]

In [6]:
ks = [2,64]
splits = ["dev","test"]
num_replacements = [24,32,48,64] # [1,2,3,4,6,8,10,12,16]
datasets = []

import json, random
blimp_datasets = []
for k in ks:
    parser = make_dyck_parser(k)
    vocabulary = get_vocab_of_bracket_types(k)[1]
    for split in splits:
        dataset = f"data/dyckkm/dyckkm_k{k}_m7_100000000.{split}.json"
        texts, _ = load_gold_data(dataset, bos_tok="", eos_tok="")
        for num_repl in num_replacements: 
            print(k, num_repl, split)
            
            good, bad = create_type_mismatch_data(num_repl, texts, parser, vocabulary)
            with open(f"data/blimpfordyck/dyck_k{k}/dyck_k{k}_{split}_blimpdycktypemismatch_{num_repl}.jsonl", "w") as f:
                for g,b in zip(good, bad):
                    f.write(json.dumps({
                        "sentence_good": g,
                        "sentence_bad": b
                    }))
                    f.write("\n")

2 24 dev


100%|██████████| 3895/3895 [00:01<00:00, 2152.06it/s]


2 32 dev


100%|██████████| 3895/3895 [00:01<00:00, 2535.86it/s]


2 48 dev


...

100%|██████████| 2750/2750 [00:01<00:00, 1535.18it/s]


64 48 test


100%|██████████| 2750/2750 [00:01<00:00, 1897.33it/s]


64 64 test


100%|██████████| 2750/2750 [00:01<00:00, 2389.66it/s]


In [9]:
splits = ["dev","test"]
num_replacements = [64]# [24,32,48,64]# [4,6,8,10,12,16] #[1,2,3]#,4,6,8,10,12,16]
datasets = []

import json, random

for split in splits:
    dataset = f"data/dyckkm/underspec_100000000.{split}.json"
    texts, _ = load_gold_data(dataset, bos_tok="", eos_tok="")
    for num_repl in num_replacements: 
        print(num_repl, split)
        good, bad = create_type_mismatch_u_data(num_repl, texts)
        with open(f"data/blimpfordyck/dyck_u/dyck_u_{split}_blimpdycktypemismatch_{num_repl}.jsonl", "w") as f:
            for g,b in zip(good, bad):
                f.write(json.dumps({
                    "sentence_good": g,
                    "sentence_bad": b
                }))
                f.write("\n")

64 dev


100%|██████████| 4132/4132 [00:01<00:00, 2562.45it/s]


64 test


100%|██████████| 2485/2485 [00:04<00:00, 568.35it/s]


In [ ]:
# break bracketing
def create_type_breakbracketing_data(distance, texts, parser):
    results = []
    for sent in tqdm.tqdm(texts):
        sentsplit = sent.split()
        if len(sentsplit) < distance + 2:
            continue
        success = False
        max_tries = 20
        num_tries = 0
        while not success:
            sentsplit = sent.split()
            
            ix_to_replace = random.choice(range(len(sentsplit) - distance))
            old_token_first = sentsplit[ix_to_replace]
            old_token_second = sentsplit[ix_to_replace+distance]

            sentsplit[ix_to_replace] = old_token_second
            sentsplit[ix_to_replace+distance] = old_token_first

            if len(list(parser.parse(sentsplit))) == 0:
                success = True
                results.append((sent, ' '.join(sentsplit)))
            if not success:
                num_tries += 1 
                if num_tries >= max_tries:
                    success=True
    return [res[0] for res in results], [res[1] for res in results]



In [59]:
ks = [1,2,64]
splits = ["dev","test"]
distances = [48,64]# [1,2,3,4,6,8,10,12,16,24,32]
datasets = []

import json, random
blimp_datasets = []
for k in ks:
    parser = make_dyck_parser(k)
    vocabulary = get_vocab_of_bracket_types(k)[1]
    for split in splits:
        dataset = f"data/dyckkm/dyckkm_k{k}_m7_100000000.{split}.json"
        texts, _ = load_gold_data(dataset, bos_tok="", eos_tok="")
        for distance in distances: 
            print(k, distance, split)
            
            good, bad = create_type_breakbracketing_data(distance, texts, parser)
            with open(f"data/blimpfordyck/dyck_k{k}/dyck_k{k}_{split}_blimpdyckbrokenbracketing_{distance}.jsonl", "w") as f:
                for g,b in zip(good, bad):
                    f.write(json.dumps({
                        "sentence_good": g,
                        "sentence_bad": b
                    }))
                    f.write("\n")

1 48 dev


100%|██████████| 3925/3925 [00:08<00:00, 462.15it/s]


1 64 dev


100%|██████████| 3925/3925 [00:03<00:00, 1141.47it/s]


1 48 test


...

100%|██████████| 3946/3946 [00:00<00:00, 5064.83it/s]


64 48 test


100%|██████████| 2750/2750 [00:02<00:00, 1092.15it/s]


64 64 test


100%|██████████| 2750/2750 [00:02<00:00, 1298.39it/s]


In [60]:
splits = ["dev", "test"]
distances = [1,2,3,4,6,8,10,12,16,24,32,48,64]
parser = make_dycku_grammar()[1]
for split in splits:
    dataset = f"data/dyckkm/underspec_100000000.{split}.json"
    texts, _ = load_gold_data(dataset, bos_tok="", eos_tok="")
    for distance in distances: 
        print(distance, split)
        good, bad = create_type_breakbracketing_data(distance, texts, parser)
        with open(f"data/blimpfordyck/dyck_u/dyck_u_{split}_blimpdyckbrokenbracketing_{distance}.jsonl", "w") as f:
            for g,b in zip(good, bad):
                f.write(json.dumps({
                    "sentence_good": g,
                    "sentence_bad": b
                }))
                f.write("\n")

1 dev


100%|██████████| 4132/4132 [00:30<00:00, 137.34it/s]


1 dev


100%|██████████| 4132/4132 [00:30<00:00, 135.15it/s]


1 dev


...

100%|██████████| 2485/2485 [00:13<00:00, 187.16it/s]


1 test


100%|██████████| 2485/2485 [00:11<00:00, 215.22it/s]


1 test


100%|██████████| 2485/2485 [00:11<00:00, 223.96it/s]
